In [17]:
#STEP 1: importing all needed moduels

import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt
import skmisc

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell
from dask.diagnostics import ProgressBar
from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase

#STEP 1+: Set maximum number of jobs
sc.settings.njobs = 32

Divide each file into by states and then running each state for network reconstruction
- A is patient pretreatment pdx
- B is patient posttreatment pdx

In [18]:
#prepping
DATASET_ID = 'lx599'

In [33]:
#STEP 1++: preparing pathway constant variables for easy coding

SOHRAB_RESOURCES_FOLDERNAME = "/work/shah/users/salehis/projects/cdm/data/sclc/{}".format(DATASET_ID)
RESULTS_FOLDERNAME = "/home/linl5/project/SCLC/results/{}".format(DATASET_ID)
FIGURES_FOLDERNAME = "/home/linl5/project/SCLC/figures"
AUXILLIARIES_FOLDERNAME = "/home/linl5/project/SCLC/auxilliaries"
RESOURCES_FOLDERNAME = "/home/linl5/project/SCLC/resources"
DATA_FOLDERNAME = "/home/linl5/project/SCLC/data/{}".format(DATASET_ID)

In [34]:
#Downloaded fromm pySCENIC github repo: https://github.com/aertslab/pySCENIC/tree/master/resources Aug-1-2023
#lambert2018.txt used in their cancer patient tutorial
HUMAN_TFS_FNAME = os.path.join(RESOURCES_FOLDERNAME, 'allTFs_hg38.txt')

In [35]:
#STEP 2: Downloading metadata and expression matrix
#input: expression matrix and metadata file

#organized by cell ID and Gene matrix, values are counts of read in that cell
COUNTS_MTX_FNAME = os.path.join(SOHRAB_RESOURCES_FOLDERNAME, 'rna.h5ad')

In [36]:
#STEP 3: Importing and Analyzing the rna DATA

# Read the H5AD file using anndata
adata = ad.read(COUNTS_MTX_FNAME)

In [23]:
#STEP4: Understanding Dataset

#print out information about anndata
print(adata)

#print out shape
print("Shape of expression matrix: ", adata.shape)

# the columns (variables) of the expression matrix
print("Columns (variables):")
print(adata.var)

# the rows (observations) of the expression matrix
print("\nRows (observations):")
print(adata.obs)


AnnData object with n_obs × n_vars = 33207 × 33538
    obs: 'timepoint', 'datatag', 'batch', 'state', 'cna_name', 'is_in_rna'
    var: 'gene_ids', 'feature_types'
Shape of expression matrix:  (33207, 33538)
Columns (variables):
                    gene_ids    feature_types
MIR1302-2HG  ENSG00000243485  Gene Expression
FAM138A      ENSG00000237613  Gene Expression
OR4F5        ENSG00000186092  Gene Expression
AL627309.1   ENSG00000238009  Gene Expression
AL627309.3   ENSG00000239945  Gene Expression
...                      ...              ...
AC233755.2   ENSG00000277856  Gene Expression
AC233755.1   ENSG00000275063  Gene Expression
AC240274.1   ENSG00000271254  Gene Expression
AC213203.1   ENSG00000277475  Gene Expression
FAM231C      ENSG00000268674  Gene Expression

[33538 rows x 2 columns]

Rows (observations):
                              timepoint datatag batch state  \
AAACCCAAGACGGTTG-1_Lx599_a-UU       UUa   Lx599     0   UUa   
AAACCCAAGAGTTGTA-1_Lx599_a-UU       UUa   Lx59

In [24]:
#STEP 5: preprocessing and filtering

#make gene name unique
adata.var_names_make_unique()

#processing out data-prefilter
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

#saving a copy of the power filtered count into raw
adata.raw = adata

#post filer: previous was n_obs × n_vars = 33207 × 33538
print("Post Filter: ", adata.shape)


Post Filter:  (33108, 28701)


In [25]:
# STEP 6: Prepping for timepoint splitting
adata.layers['counts'] = adata.X
adata.raw = adata


In [26]:
#STEP 7: subsampling time point (IDEA-> in original sample filte by hvg, then split the following into states, then cluster and subsample from cluster)

# set the n_top_genes as necessary
sc.pp.highly_variable_genes(adata, n_top_genes=900, subset=True, flavor='seurat_v3')

#Splitting the cells by states
unique_state = adata.obs['state'].unique()
print(unique_state)

#storing
adata_by_state = {}

#selecting out by TP
for state in unique_state:
    adata_subset = adata[adata.obs['state'] == state].copy()
    adata_by_state[state] = adata_subset
    print(state, ": ", adata_by_state[state].shape)

    

['UUa', 'UTb', 'UUb', 'UTa']
Categories (4, object): ['UTa', 'UTb', 'UUa', 'UUb']
UUa :  (7280, 900)
UTb :  (5285, 900)
UUb :  (10036, 900)
UTa :  (10507, 900)


The Leiden algorithm improves upon Louvain by using the "agglomerative" approach to optimize a different quality function known as the "improved modularity." Improved modularity has a resolution parameter that allows Leiden to control the granularity of the clustering solution. It also uses a smart local move algorithm to avoid some of the resolution limit issues present in Louvain. Higher paramter means more identified communities. 

#The Leiden algorithm is a hierarchical clustering algorithm, that recursively merges communities into single nodes by greedily optimizing the modularity and the process repeats in the condensed graph.
#The Leiden algorithm improves upon Louvain by using the "agglomerative" approach to optimize a different quality function known as the "improved modularity." Improved modularity has a resolution parameter that allows Leiden to control the granularity of the clustering solution. It also uses a smart local move algorithm to avoid some of the resolution limit issues present in Louvain.

In [85]:
#STEP 8: cluster definition

def cluster_rna(bdata):
    #I have already did filtering and HVG selection on main anndata, we want to keep same hvg across our states
    sc.pp.normalize_total(bdata)    
    sc.pp.log1p(bdata)
    sc.pp.pca(bdata)
    sc.pp.neighbors(bdata)
    sc.tl.umap(bdata)
    sc.tl.leiden(bdata, resolution=2)
    return bdata

In [86]:
#STEP 9: Clustering

for state in unique_state:
    adata_by_state[state] = cluster_rna(adata_by_state[state])

In [114]:
#STEP 10: Subsampling

from tqdm import tqdm
frac_cells = 0.07
data_sub = {}

#Consistent randomization
np.random.seed(0)

for state in unique_state:
    sub_cells = []
    for clust in tqdm(adata_by_state[state].obs['leiden'].unique()):
        # sample 10% of cells from each cluster
        cells_in_clust = adata_by_state[state].obs_names[adata_by_state[state].obs['leiden'] == clust].copy()
        #dropping out clusters with less than 5 cells
        if (len(cells_in_clust) > 5):
            chosen_cells = np.random.choice(cells_in_clust, size=int(len(cells_in_clust)*frac_cells), replace=False)
            sub_cells.extend(chosen_cells)
    data_sub[state] = adata_by_state[state][sub_cells, :].copy()

100%|██████████| 23/23 [00:00<00:00, 3743.75it/s]


In [117]:
#checking for subsampling population

for state in unique_state:
    print(state, data_sub[state].shape)
    set1 = set(data_sub[state].var_names)
    set2 = set(data_sub["UUa"].var_names)
    print("Same Gene set check: ", state, "UUa ", len(set1.intersection(set2)))

UUa (499, 900)
Same Gene set check:  UUa UUa  900
UTb (357, 900)
Same Gene set check:  UTb UUa  900
UUb (693, 900)
Same Gene set check:  UUb UUa  900
UTa (723, 900)
Same Gene set check:  UTa UUa  900


In [119]:
#STEP 11: copying over raw count 

for state in unique_state:
    data_sub[state].X = data_sub[state].layers['counts'].copy()
    
#check for sample output
data_sub["UUa"].to_df()

,ISG15,SLC2A5,SPSB1,NPPB,TNFRSF8,KAZN,CDA,RAP1GAP,SFN,IFI6,...,MIR548XHG,LINC02573,NCAM2,LINC01695,KCNJ6,PCP4,DSCAM,BACE2,MX2,CSTB
CACTAAGAGACATATG-1_Lx599_a-UU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0
GTGCAGCAGGACGCAT-1_Lx599_a-UU,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0
AGATCCATCATCACAG-1_Lx599_a-UU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
ACCGTTCGTGACCTGC-1_Lx599_a-UU,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
ATCGATGAGTGCGTCC-1_Lx599_a-UU,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTCCTCACACTACGGC-1_Lx599_a-UU,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
TTTAGTCTCGACACTA-1_Lx599_a-UU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0
CTATAGGGTACAATAG-1_Lx599_a-UU,0.0,0.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAGCGTTCACAACCGC-1_Lx599_a-UU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
#STEP 12: output Timepoint specific count matrix after subsampling by state
for state in unique_state:
    EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.{}.qc.tpm.csv'.format(DATASET_ID, state))
    data_sub[state].to_df().to_csv(EXP_MTX_QC_FNAME, index=False, sep='\t')
    print("Finish with: ", EXP_MTX_QC_FNAME, " Shape: ", data_sub[state].shape)


Finish with:  /home/linl5/project/SCLC/results/lx599.UUa.qc.tpm.csv  Shape:  (499, 900)
Finish with:  /home/linl5/project/SCLC/results/lx599.UTb.qc.tpm.csv  Shape:  (357, 900)
Finish with:  /home/linl5/project/SCLC/results/lx599.UUb.qc.tpm.csv  Shape:  (693, 900)
Finish with:  /home/linl5/project/SCLC/results/lx599.UTa.qc.tpm.csv  Shape:  (723, 900)


In [14]:
# STEP 13: prepping for GRN, Loading in expression matrix and TF files

#Loading TF
tf_names = load_tf_names(HUMAN_TFS_FNAME)
print(HUMAN_TFS_FNAME, ": Size of TF list", len(tf_names))

#expression matrix
for state in unique_state:
    EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.{}.qc.tpm.csv'.format(DATASET_ID, state))
    ex_matrix = pd.read_csv(EXP_MTX_QC_FNAME, sep='\t', header=0, index_col=0)
   
    #Input Checking 
    print("\nExpression matrix shape for", DATASET_ID, state, ex_matrix.shape)

    #STEP 14: Running GRNBOOST2 for coexpression modules
    adjacencies = grnboost2(expression_data=ex_matrix, tf_names=tf_names, verbose=True)
    print("\nCOMPLETED GRNBOOST2 RUNNING FOR", DATASET_ID, state)
    print("\n", adjacencies.head())
    ADJACENCIES_FNAME = os.path.join(DATA_FOLDERNAME, "{}.{}.adjacencies.tsv".format(DATASET_ID, state))
    adjacencies.to_csv(ADJACENCIES_FNAME, index=False, sep='\t')
    print("SUCCESSFUL WRITING TO", ADJACENCIES_FNAME, "\n")
    

/home/linl5/project/SCLC/resources/allTFs_hg38.txt : Size of TF list 1892

Expression matrix shape for lx599 UUa (499, 899)
preparing dask client
parsing input
creating dask graph
8 partitions
computing dask graph
shutting down client and local cluster
finished

COMPLETED GRNBOOST2 RUNNING FOR lx599 UUa

        TF   target  importance
24  ANXA1   TM4SF1  118.565309
41   ATF5    UBE2C   98.453426
41   ATF5    CENPF   97.341648
39  TRIB3     ATF5   89.687075
24  ANXA1  S100A10   85.079732
SUCCESSFUL WRITING TO /home/linl5/project/SCLC/data/lx599.UUa.adjacencies.tsv 


Expression matrix shape for lx599 UTb (357, 899)
preparing dask client
parsing input
creating dask graph
8 partitions
computing dask graph
shutting down client and local cluster
finished

COMPLETED GRNBOOST2 RUNNING FOR lx599 UTb

        TF   target  importance
24  ANXA1  S100A11   83.944753
24  ANXA1  S100A10   79.913382
24  ANXA1    KRT19   79.809325
24  ANXA1     CD55   71.257329
24  ANXA1     EMP1   68.539123
SUCCESSF

In [14]:
#STEP 14: Prepping for RCistarget: Loading Database
DATABASE_FOLDER = "/home/linl5/project/SCLC/auxilliaries/"
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "hg38_*.mc9nr.genes_vs_motifs.rankings.feather")

db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

[FeatherRankingDatabase(name="hg38__refseq-r80__10kb_up_and_down_tss.mc9nr.genes_vs_motifs.rankings"),
 FeatherRankingDatabase(name="hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.genes_vs_motifs.rankings")]

In [15]:
#STEP 15: Prepping for Running RCistarget

MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDERNAME,"motifs-v9-nr.hgnc-m0.001-o0.0.tbl")


In [30]:
#STEP 16: Running RCistarget

for state in unique_state:
    #reading necessary files for rcistarget
    ADJACENCIES_FNAME = os.path.join(DATA_FOLDERNAME, "{}.{}.adjacencies.tsv".format(DATASET_ID, state))
    adjacencies = pd.read_csv(ADJACENCIES_FNAME, sep='\t')
    print("\nFINISHED READING ADJACENCIES FILE", ADJACENCIES_FNAME,"\n")
    EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.{}.qc.tpm.csv'.format(DATASET_ID, state))
    ex_matrix = pd.read_csv(EXP_MTX_QC_FNAME, sep='\t', header=0, index_col=0)
    print("\nFINISHED READING EXPRESSION MATRIX", EXP_MTX_QC_FNAME,"\n")
    MODULES_FNAME = os.path.join(DATA_FOLDERNAME, '{}.{}.modules.p'.format(DATASET_ID, state))
    MOTIFS_FNAME = os.path.join(DATA_FOLDERNAME, '{}.{}.motifs.csv'.format(DATASET_ID, state))
    REGULONS_FNAME = os.path.join(DATA_FOLDERNAME, '{}.{}.regulons.p'.format(DATASET_ID, state))

    #making modules from adjacencies
    modules = list(modules_from_adjacencies(adjacencies, ex_matrix))
    
    #writing modules object to file
    with open(MODULES_FNAME, 'wb') as f:
        pickle.dump(modules, f)
    print("\nCOMPLETED COEXPRESSION MODULE WRITING:", MODULES_FNAME,"\n")
    
    #running Rcistarget with progress bar: searching for enriched motifs and true candidate genes
    with ProgressBar():
        df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME, client_or_address="dask_multiprocessing") 
    
    #writing enriched motifs with candidate target to file
    df.to_csv(MOTIFS_FNAME)
    print("\nCOMPLETED WRITING ENRICHED MOTIFS", MOTIFS_FNAME,"\n")
    print(df.head())
    
    #making regulon objects
    regulons = df2regulons(df)
    
    #writing regulon objects to file
    with open(REGULONS_FNAME, 'wb') as f:
        pickle.dump(regulons, f)
    print("\nCOMPLETED WRITING DISCOVERED REGULON", REGULONS_FNAME,"\n")


2023-08-03 13:01:00,443 - pyscenic.utils - INFO - Calculating Pearson correlations.

2023-08-03 13:01:00,448 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [False].



FINISHED READING ADJACENCIES FILE /home/linl5/project/SCLC/data/lx599.UUa.adjacencies.tsv 


FINISHED READING EXPRESSION MATRIX /home/linl5/project/SCLC/results/lx599.UUa.qc.tpm.csv 




2023-08-03 13:01:00,669 - pyscenic.utils - INFO - Creating modules.
/home/linl5/miniconda3/envs/pyscenic/lib/python3.10/site-packages/pyscenic/utils.py:244: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df = adjacencies.groupby(by=COLUMN_NAME_TARGET).apply(



COMPLETED COEXPRESSION MODULE WRITING: /home/linl5/project/SCLC/data/lx599.UUa.modules.p 

[                                        ] | 0% Completed | 29.26 sms


2023-08-03 13:01:33,346 - pyscenic.transform - WARNING - Less than 80% of the genes in Regulon for MEOX2 could be mapped to hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.genes_vs_motifs.rankings. Skipping this module.


...

[                                        ] | 0% Completed | 120.75 s


2023-08-03 13:10:18,968 - pyscenic.transform - WARNING - Less than 80% of the genes in IL24 could be mapped to hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.genes_vs_motifs.rankings. Skipping this module.


[                                        ] | 0% Completed | 122.38 s


2023-08-03 13:10:20,646 - pyscenic.transform - WARNING - Less than 80% of the genes in NPAS2 could be mapped to hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.genes_vs_motifs.rankings. Skipping this module.


[########################################] | 100% Completed | 132.97 s
COMPLETED WRITING ENRICHED MOTIFS /home/linl5/project/SCLC/data/lx599.UTa.motifs.csv 

                                             Enrichment            \
                                                    AUC       NES   
TF     MotifID                                                      
EGR3   transfac_pro__M01572                    0.106552  3.271255   
PKNOX2 transfac_pro__M01549                    0.167524  3.123434   
DDIT3  cisbp__M5988                            0.117700  4.648830   
       taipale__Atf4_DBD_NGGATGATGCAATM_repr   0.113290  4.410297   
       cisbp__M5292                            0.101619  3.779085   

                                                                    \
                                             MotifSimilarityQvalue   
TF     MotifID                                                       
EGR3   transfac_pro__M01572                               0.000000   
PKNOX2 tr

After grouping the data by timepoints
- How is clustering in this case difference by just making anndata subset selected by timepoint (pin)
- Do i do PCA after clustering -> pca changes the expression matrix so output to GENIE3 is not pca, just raw unnormalize, unpertrude data except basic filtering
- If you have more than one condition, it’s often helpful to perform integration to align the cells -? and then within each timepoint there would be three batches so i should find a way to remove this right